# Packages

In [121]:
using FinEtools, FinEtoolsAcoustics, LinearAlgebra, StatsBase, HDF5
cd(desktopPath) # change directory

# Initial setup

In [122]:
rho = 1.21 * phun("kg/m^3") # mass density
c  = 343.0 * phun("m/s") # sound speed
bulk =  c ^ 2 * rho
frequency = 500.0 # frequency of the incident wave, Hz
omega = 2 * pi * frequency
dt = 1.0 / frequency / 80
tfinal = 1005 * dt
nsteps = round(Int, tfinal / dt) + 1
println("Step takes $(sciNotation(dt, 3)) s")
println("Total of $(round(Int, nsteps)) steps")
println("Final time: $(sciNotation(tfinal, 3)) s")

Step takes 2.5E-5 s
Total of 1006 steps
Final time: 2.512E-2 s


# Mesh

Geometry

In [123]:
# list of meshes
Meshes = Array{Tuple{FENodeSet, AbstractFESet}, 1}()
# Q4 discretization of large rectangular domain divisions
push!(Meshes, Q4quadrilateral([0.0 0.0; 1.2 2.6], 10, 26)) # 1
push!(Meshes, Q4quadrilateral([0.0 2.6; 1.6 5.45], 13, 30)) # 2
push!(Meshes, Q4quadrilateral([1.35 0.0; 4.05 2.6], 25, 24)) # 3
push!(Meshes, Q4quadrilateral([1.6 2.6; 4.05 5.8], 23, 34)) # 4
push!(Meshes, Q4quadrilateral([4.25 1.2; 5.25 4.5], 10, 35)) # 5
push!(Meshes, Q4quadrilateral([4.05 4.5; 5.25 5.3], 12, 9)) # 6
push!(Meshes, Q4quadrilateral([4.25 5.3; 5.25 5.85], 10, 4)) # 7
push!(Meshes, Q4quadrilateral([5.25 1.2; 7.0 3.92], 22, 29)) # 8
# merge meshes
fens, outputfes = mergenmeshes(Meshes, 0.05)
# concatenate connectivities
fes = cat(outputfes[5], cat(outputfes[6], cat(outputfes[7], outputfes[8])))
fes = cat(outputfes[1], cat(outputfes[2], cat(outputfes[3], cat(outputfes[4], fes))))
println("$(size(fens.xyz, 1)) nodes")
println("$(size(fes.conn, 1)) elements")
println("$(round(Int, size(fens.xyz, 1) * nsteps)) dataset points (using Q4)")

3362 nodes
3168 elements
3382172 dataset points (using Q4)


Subdomain selection

In [124]:
# Identify boundary finite element set
bfes = meshboundary(fes)
# In case there are any unconnected nodes, remove them, and renumber the elements.
connected = findunconnnodes(fens, fes)
fens, new_numbering = compactnodes(fens, connected)
fess = renumberconn!(fes, new_numbering)
# The geometry and the solution (pressure) fields
geom = NodalField(fens.xyz)
P = NodalField(zeros(FCplxFlt, size(fens.xyz, 1), 1));

Visualize domain

In [125]:
# Export three VTK files: one for the interior of the fluid, and one for the boundary
vtkexportmesh("interior.vtk", fes.conn, fens.xyz, FinEtools.MeshExportModule.VTK.Q4);

# Discrete model

Setup

In [126]:
# Number the degrees of freedom in the pressure field.
numberdofs!(P)
# Create the finite element machine for the fluid.
material = MatAcoustFluid(bulk, rho)
femm = FEMMAcoust(IntegDomain(fes, GaussRule(3, 2)), material)
# Use the machine calculate the acoustic stiffness and mass matrices.
S = acousticstiffness(femm, geom, P)
C = acousticmass(femm, geom, P)
# "damping" matrix for the absorbing boundary conditions (ABCs)
D = acousticABC(
  FEMMAcoustSurf(IntegDomain(bfes, GaussRule(2, 2)), material), geom, P
)
loadEle = subset(fes, selectelem(fens, fes, withnodes = [1984, 1985, 1961, 1962]))
dipfemm = FEMMAcoustSurf(IntegDomain(loadEle, GaussRule(2, 2)), material)
# initial condition as load in nodes of an element
function pointLoad(dpdn, xyz, J, label, t)::Float64
  dpdn[1] = t == dt ? -rho * 0.02 : 0.0
end;

Solution

In [ ]:
# Solve the transient acoustics equations.
# The loop executes inside this local scope
# initialize variable for dataset.
# dataset is matrix with pressures value. It has (number of nodes)
# rows and (number of timesteps) columns
pressureMat = zeros(Float32, (nnodes(P), nsteps))
P1 = let
  t = 0.0 # Initial time
  P0 = deepcopy(P)
  P0.values .= 0.0 # initially all pressure is zero
  vP0 = gathersysvec(P0)
  vQ0 = zeros(eltype(vP0), size(vP0))
  # The `P1` field will be the output of this computation:
  # the final value of the pressure field
  P1 = deepcopy(P0)
  fi = ForceIntensity( # Initial load
    FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
  )
  La0 = distribloads(dipfemm, geom, P1, fi, 2)
  A = (2.0 / dt) * S + D + (dt / 2.0) * C
  step = 0
  while t <= tfinal
    step += 1
    t += dt
    # Update load
    fi = ForceIntensity( # Initial load
      FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
    )
    La1 = distribloads(dipfemm, geom, P1, fi, 2)
    # Solve for the rate of the pressure
    vQ1 = A \ ((2 / dt) * (S * vQ0) - D * vQ0 - C * (2 * vP0 + (dt / 2) * vQ0) + La0 + La1)
    # Update pressure field
    vP1 = vP0 + (dt / 2) * (vQ0 + vQ1)
    # Swap variables for next step  
    vP0 = deepcopy(vP1)
    vQ0 = deepcopy(vQ1)
    P1 = scattersysvec!(P1, vec(vP1))
    P0 = deepcopy(P1)
    La0 = deepcopy(La1)
    # include current pressure field in dataset.
    global pressureMat[:, step] .= real.(P1.values)
  end
  P1 # Return the final pressure
end;

Export pressure fields

In [128]:
# create folder and save vtk files with
# pressure fields in all steps
randomFolderName = rand(0:9999999)
mkpath("./vtkFiles/$randomFolderName")
for fileID in axes(pressureMat, 2)
  vtkexportmesh(
    "./vtkFiles/$randomFolderName/$randomFolderName-$fileID.vtk",
    fes.conn, geom.values, FinEtools.MeshExportModule.VTK.Q4;
    scalars = [( "pressure", pressureMat[:, fileID] |> vec)]
  )
end;

# Data

Summary statistics

In [129]:
statsum(pressureMat)

Summary Stats:
Length:         3382172
Missing Count:  0
Mean:           -0.000000
Minimum:        -0.000002
1st Quartile:   -0.000000
Median:         -0.000000
3rd Quartile:   -0.000000
Maximum:        0.000000
Standard deviation: 4.8653E-8
3374790/3382172 (99.8%) non-zero elements.



Normalize

In [130]:
# standardize
standardData = (pressureMat .- mean(pressureMat)) ./ std(pressureMat)
# unit normalization
unitData = (pressureMat .- minimum(pressureMat)) ./ maximum(abs.(pressureMat));

Check normalizations with summary statistics

In [131]:
println("Standardization\n")
statsum(standardData)
println("Unit normalization\n")
statsum(unitData)

Standardization



Summary Stats:
Length:         3382172
Missing Count:  0
Mean:           0.000000
Minimum:        -42.340023
1st Quartile:   -0.225567
Median:         0.132616
3rd Quartile:   0.564643
Maximum:        6.775985
Standard deviation: 1.0E0
3382172/3382172 (100.0%) non-zero elements.

Unit normalization



Summary Stats:
Length:         3382172
Missing Count:  0
Mean:           0.986840
Minimum:        0.000000
1st Quartile:   0.981582
Median:         0.989931
3rd Quartile:   1.000000
Maximum:        1.144771
Standard deviation: 2.3307E-2
3382171/3382172 (100.0%) non-zero elements.



Save in HDF5

In [132]:
# create dataset files
for (data, name) in zip(
  [pressureMat, standardData, unitData],
  ["OpInfData", "standardData", "unitData"]
)
  h5open("./vtkFiles/OpInfData/$name.h5", "w") do dataFile
    # [p₁ p₂ ... pₜ] evolution of pressure field
    dataFile[name] = data
  end;
end

Summary statistics to check files

In [133]:
for name in ["OpInfData", "standardData", "unitData"]
  h5open("./vtkFiles/OpInfData/$name.h5", "r") do f
    for field in HDF5.get_datasets(f)
      @show HDF5.name(field)[2 : end]
      d = HDF5.read(field)
      println("size $(size(d))")
      statsum(d)
      println()
    end
  end
end

(HDF5.name(field))[2:end] = "OpInfData"
size (3362, 1006)


Summary Stats:
Length:         3382172
Missing Count:  0
Mean:           -0.000000
Minimum:        -0.000002
1st Quartile:   -0.000000
Median:         -0.000000
3rd Quartile:   -0.000000
Maximum:        0.000000
Standard deviation: 4.8653E-8
3374790/3382172 (99.8%) non-zero elements.


(HDF5.name(field))[2:end] = "standardData"
size (3362, 1006)


Summary Stats:
Length:         3382172
Missing Count:  0
Mean:           0.000000
Minimum:        -42.340023
1st Quartile:   -0.225567
Median:         0.132616
3rd Quartile:   0.564643
Maximum:        6.775985
Standard deviation: 1.0E0
3382172/3382172 (100.0%) non-zero elements.


(HDF5.name(field))[2:end] = "unitData"
size (3362, 1006)


Summary Stats:
Length:         3382172
Missing Count:  0
Mean:           0.986840
Minimum:        0.000000
1st Quartile:   0.981582
Median:         0.989931
3rd Quartile:   1.000000
Maximum:        1.144771
Standard deviation: 2.3307E-2
3382171/3382172 (100.0%) non-zero elements.


